In [2]:
# !pip install -q Bio pandas tqdm torch
# ! pip install torch 
# --index-url https://download.pytorch.org/whl/gpu
!hostname
!nvidia-smi
# !cat /usr/local/cuda/version.txt



cn-0042
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
import pandas as pd
from tqdm import tqdm
import time
from Bio import SeqIO
import requests
import json
import numpy as np
import io 
import base64
import torch

SCRATCH_DIR = '/gpfs/scratch/petera17'
DATASETS_DIR = f'{SCRATCH_DIR}/scratch/datasets'

In [ ]:
torch.cuda.is_available()

In [ ]:
# Read Clinvar
clinvar_df = pd.read_csv("/gpfs/scratch/petera17/scratch/datasets/ClinVar_benchmark_all_PB.csv")

# Read Fasta
fasta_file = f"{DATASETS_DIR}/genome.fa"
parsed_fasta_iter = SeqIO.parse(fasta_file, "fasta")
record_dict = {}
try:
    while(True):
        seq_req = next(parsed_fasta_iter)
        record_dict[seq_req.id] = seq_req
except Exception as e:
    print(e)

seq_dict = {}
for i in list(range(1,23)) + ['Y','X']:
  seq_dict[str(i)] = record_dict[f'chr{i}']

# Append Fasta seq to clinvar
window = 4000
clinvar_df["SEQ"] = clinvar_df.apply(
    lambda x: "".join(
        seq_dict[x['CHROM']].seq[int(x['POS'])-window:int(x['POS'])+window])
    , axis=1)

#Save
# clinvar_df.to_csv(f'{DATASETS_DIR}/coding_with_seq_df.csv', index=False)

def replace_3999th_char(seq, alt):
    if pd.isna(seq) or pd.isna(alt) or len(seq) < 3999:
        return seq  # return unchanged if too short or missing
    return seq[:3999] + alt + seq[4000:]

# Apply the function row-wise
clinvar_df['ALT_SEQ'] = clinvar_df.apply(lambda row: replace_3999th_char(row['SEQ'], row['ALT']), axis=1)

In [5]:
clinvar_df = pd.read_csv(f'{DATASETS_DIR}/clinvar_w_seq_202504.csv')
clinvar_df.shape

(259617, 30)

In [42]:
amino_acids = {'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I',
               'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'}

coding_df = clinvar_df[clinvar_df["ClinVarName_coding_sequence"] == 1]

non_coding_df = clinvar_df[clinvar_df["ClinVarName_coding_sequence"] == 0]

def classify_variant(row):
    if row['ClinVarName_AAREF'] != row['ClinVarName_AAALT'] and row['ClinVarName_AAALT'] in amino_acids:
        return 'missense'
    if row['ClinVarName_AAALT'] == '*':
        return 'stop_gain'
    if row['ClinVarName_AAALT'] == "=":
        return 'synonymous'
    return 'other'

coding_df['variant_category'] = coding_df.apply(classify_variant, axis=1)
non_coding_df['variant_category'] = "non_coding"

/tmp/ipykernel_3116804/2617188657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coding_df['variant_category'] = coding_df.apply(classify_variant, axis=1)
/tmp/ipykernel_3116804/2617188657.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_coding_df['variant_category'] = "non_coding"


In [58]:
import pandas as pd

# Binary flags to check in non_coding_df
non_coding_flags = [
    'five_prime_UTR',
    'three_prime_UTR',
    'mRNA_intron',
    'mRNA_splice',
    'snRNA',
    'snRNA_exon',
    'snoRNA',
    'snoRNA_exon',
]

# Star levels (exact match)
star_levels = [0, 1, 2, 3, 4]
columns = ['Total', 'NaN stars'] + [f'{s} stars' for s in star_levels]

# Helper function
def get_benign_pathogenic_string(df):
    benign = (df['INFO'] == 0).sum()
    pathogenic = (df['INFO'] == 1).sum()
    return f"{pathogenic} / {benign}"

# Build result
result_nc = {}

for flag in non_coding_flags:
    row = {}

    flagged_df = non_coding_df[non_coding_df[flag] == 1]

    # Total
    row['Total'] = get_benign_pathogenic_string(flagged_df)

    # NaN stars
    nan_df = flagged_df[flagged_df['ClinVar_gold_stars'].isna()]
    row['NaN stars'] = get_benign_pathogenic_string(nan_df)

    # Exact star levels
    for s in star_levels:
        star_df = flagged_df[flagged_df['ClinVar_gold_stars'] == s]
        row[f'{s} stars'] = get_benign_pathogenic_string(star_df)

    result_nc[flag] = row

# Create DataFrame
summary_nc_df = pd.DataFrame.from_dict(result_nc, orient='index', columns=columns)
summary_nc_df.index.name = 'non_coding_category'

# Display
print(summary_nc_df)

                             Total   NaN stars 0 stars       1 stars  \
non_coding_category                                                    
five_prime_UTR           68 / 2463     35 / 88   0 / 0      20 / 835   
three_prime_UTR         66 / 12138     30 / 95   0 / 0     22 / 5523   
mRNA_intron          12438 / 87242  1932 / 885   0 / 0  7504 / 43184   
mRNA_splice             11001 / 77    1492 / 6   0 / 0     6809 / 38   
snRNA                       18 / 3       6 / 0   0 / 0         6 / 2   
snRNA_exon                  18 / 3       6 / 0   0 / 0         6 / 2   
snoRNA                     12 / 16      11 / 0   0 / 0         0 / 6   
snoRNA_exon                12 / 16      11 / 0   0 / 0         0 / 6   

                          2 stars     3 stars 4 stars  
non_coding_category                                    
five_prime_UTR          13 / 1518      0 / 22   0 / 0  
three_prime_UTR         14 / 6425      0 / 95   0 / 0  
mRNA_intron          2556 / 42117  439 / 1056   7 / 0  

In [44]:
# Prepare gold star thresholds
thresholds = [1, 2, 3, 4]
combined_counts = {}

# Helper function to get benign/pathogenic counts
def count_benign_pathogenic(df):
    counts = {}
    for category in df['variant_category'].unique():
        sub_df = df[df['variant_category'] == category]
        benign = (sub_df['INFO'] == 0).sum()
        pathogenic = (sub_df['INFO'] == 1).sum()
        counts[category] = f"{benign} / {pathogenic}"
    return pd.Series(counts)

# Total
combined_counts['Total'] = count_benign_pathogenic(coding_df)

# NaN stars
nan_df = coding_df[coding_df['ClinVar_gold_stars'].isna()]
combined_counts['NaN stars'] = count_benign_pathogenic(nan_df)

# Thresholds
for t in thresholds:
    filtered = coding_df[coding_df['ClinVar_gold_stars'] >= t]
    combined_counts[f'≥{t} stars'] = count_benign_pathogenic(filtered)

# Combine into DataFrame
summary_df = pd.DataFrame(combined_counts).fillna("0 / 0")

# Display
print(summary_df)


                    Total    NaN stars       ≥1 stars      ≥2 stars  \
missense    31604 / 23025  2763 / 7628  28841 / 15397  16154 / 5483   
stop_gain     151 / 43230    17 / 2980    134 / 40250    69 / 11495   
synonymous    42517 / 284    2526 / 78    39991 / 206    22347 / 98   

              ≥3 stars ≥4 stars  
missense    727 / 1197    0 / 8  
stop_gain     1 / 1772    0 / 4  
synonymous    364 / 30    0 / 0  


In [92]:
import pandas as pd

sampled_dfs = {}

# Helper: get 100 benign / 100 pathogenic samples per variant_category
def get_balanced_samples(df):
    samples = []
    for category, group in df.groupby('variant_category'):
        benign = group[group['INFO'] == 0]
        pathogenic = group[group['INFO'] == 1]
        
        if len(benign) >= 100: 
            benign_sample = benign.sample(n=100, random_state=42)
            samples.append(benign_sample)
        else:
            samples.append(benign)

        if len(pathogenic) >= 100:
            pathogenic_sample = pathogenic.sample(n=100, random_state=42)
            samples.append(pathogenic_sample)
        else:
            samples.append(pathogenic)

    if samples:
        result = pd.concat(samples).reset_index(drop=True)
        return result
    else:
        return pd.DataFrame()

# Handle exact gold star levels
# for star in gold_star_levels:
two_star_coding_df = coding_df[coding_df["ClinVar_gold_stars"] >= 2]
sampled_coding_df = get_balanced_samples(coding_df)

# Optional: inspect sample size breakdown
print(sampled_coding_df.groupby(['variant_category', 'INFO']).size())


variant_category  INFO
missense          0       100
                  1       100
stop_gain         0       100
                  1       100
synonymous        0       100
                  1       100
dtype: int64


In [59]:
import pandas as pd

non_coding_flags = [
    'five_prime_UTR',
    'three_prime_UTR',
    'mRNA_intron',
    'mRNA_splice',
    'snRNA',
    'snRNA_exon',
    'snoRNA',
    'snoRNA_exon',
]

sampled_dfs = {}

# Helper to sample 100 benign + 100 pathogenic per flag
def get_balanced_samples(df, flag):
    flagged = df[df[flag] == 1]
    benign = flagged[flagged['INFO'] == 0]
    pathogenic = flagged[flagged['INFO'] == 1]
    
    samples = []

    if len(benign) > 100:
        samples.append(benign.sample(n=100, random_state=42))
    else:
        samples.append(benign)

    if len(pathogenic) > 100:
        samples.append(pathogenic.sample(n=100, random_state=42))
    else:
        samples.append(pathogenic)

    if samples:
        combined = pd.concat(samples)
        combined['non_coding_category'] = flag
        return combined
    else:
        return pd.DataFrame()

# Sample for each non-coding flag
for flag in non_coding_flags:
    sampled = get_balanced_samples(non_coding_df, flag)
    if not sampled.empty:
        sampled_dfs[flag] = sampled

# Combine all into a single DataFrame
non_coding_samples = pd.concat(sampled_dfs.values(), ignore_index=True)

# Optional: Inspect result
print(non_coding_samples['non_coding_category'].value_counts())
print(non_coding_samples.groupby(['non_coding_category', 'INFO']).size())


non_coding_category
mRNA_intron        200
mRNA_splice        177
five_prime_UTR     168
three_prime_UTR    166
snoRNA_exon         28
snoRNA              28
snRNA_exon          21
snRNA               21
Name: count, dtype: int64
non_coding_category  INFO
five_prime_UTR       0       100
                     1        68
mRNA_intron          0       100
                     1       100
mRNA_splice          0        77
                     1       100
snRNA                0         3
                     1        18
snRNA_exon           0         3
                     1        18
snoRNA               0        16
                     1        12
snoRNA_exon          0        16
                     1        12
three_prime_UTR      0       100
                     1        66
dtype: int64


In [ ]:
sampled_coding_df

In [93]:
# Combine coding and non-coding sampled datasets
final_combined_samples = pd.concat(
    [sampled_coding_df, non_coding_samples],
    ignore_index=True
)

final_combined_samples.to_csv(f"{DATASETS_DIR}/clinvar_w_seq_202504_sampled.csv")


In [7]:
final_combined_samples = pd.read_csv(f"{DATASETS_DIR}/clinvar_w_seq_202504_sampled.csv")
final_combined_samples

,Unnamed: 0,#CHROM,POS,ID,REF,ALT,INFO,ClinVar_gold_stars,ClinVarName_refseq_ids,ClinVarName_AAPOS,...,snRNA,snRNA_exon,snoRNA,snoRNA_exon,other,MANE_transcript,SEQ,ALT_SEQ,variant_category,non_coding_category
0,0,4,42893456,1297902,G,A,0,2.0,NM_001080476.3,64.0,...,0,0,0,0,0,{'NM_001080476.3'},AAAGTTTGAAGCCAGCAGAGGGTTAGTTTGTAGGCTTAAGAAGCTA...,AAAGTTTGAAGCCAGCAGAGGGTTAGTTTGTAGGCTTAAGAAGCTA...,missense,NaN
1,1,19,12663718,712406,C,A,0,2.0,NM_000528.4,250.0,...,0,0,0,0,0,{'NM_000528.4'},TGAGGCAGGAGAATCACTAGAACCCAGGAGGCAGAGGTTGCCATGA...,TGAGGCAGGAGAATCACTAGAACCCAGGAGGCAGAGGTTGCCATGA...,missense,NaN
2,2,7,21599906,2723335,C,G,0,1.0,NM_001277115.2,929.0,...,0,0,0,0,0,{'NM_001277115.2'},GAAAAGGAAAGGCTGAGACAGTAGGAGGAAAACCAGGACAGGAAGC...,GAAAAGGAAAGGCTGAGACAGTAGGAGGAAAACCAGGACAGGAAGC...,missense,NaN
3,3,6,33435245,3004688,T,A,0,1.0,NM_006772.3,201.0,...,0,0,0,0,0,{'NM_006772.3'},GCTTGGCATCCCCCCACTTACATTCTCTTCCAGCCCCTCGGCCCCT...,GCTTGGCATCCCCCCACTTACATTCTCTTCCAGCCCCTCGGCCCCT...,missense,NaN
4,4,4,3492767,423589,C,T,0,2.0,NM_173660.5,261.0,...,0,0,0,0,0,{'NM_173660.5'},GTTTGCTCTTGCGGGTGTGGCTGTGGCTTCCCTGCCAGGGAAACTC...,GTTTGCTCTTGCGGGTGTGGCTGTGGCTTCCCTGCCAGGGAAACTC...,missense,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1404,17,8173532,929287,C,A,1,NaN,NaN,NaN,...,0,0,1,1,0,"{'NM_183065.4', 'NR_033294.2'}",AAATATTCCCCCTAGATTATCATCTATTCTTTAACTTTTGCTTATA...,AAATATTCCCCCTAGATTATCATCTATTCTTTAACTTTTGCTTATA...,non_coding,snoRNA_exon
1405,1405,17,8173550,929291,C,A,1,NaN,NaN,NaN,...,0,0,1,1,0,"{'NM_183065.4', 'NR_033294.2'}",ATCATCTATTCTTTAACTTTTGCTTATAATTTTTTGTACTATGCAT...,ATCATCTATTCTTTAACTTTTGCTTATAATTTTTTGTACTATGCAT...,non_coding,snoRNA_exon
1406,1406,17,8173550,929290,C,G,1,2.0,NaN,NaN,...,0,0,1,1,0,"{'NM_183065.4', 'NR_033294.2'}",ATCATCTATTCTTTAACTTTTGCTTATAATTTTTTGTACTATGCAT...,ATCATCTATTCTTTAACTTTTGCTTATAATTTTTTGTACTATGCAT...,non_coding,snoRNA_exon
1407,1407,17,8173586,929294,G,T,1,NaN,NaN,NaN,...,0,0,1,1,0,"{'NM_183065.4', 'NR_033294.2'}",TACTATGCATAGTTTTAACATTCTAATATATATTTAAAGTGTGTGA...,TACTATGCATAGTTTTAACATTCTAATATATATTTAAAGTGTGTGA...,non_coding,snoRNA_exon


In [13]:
from concurrent.futures import ThreadPoolExecutor

# Vortex tokenizer just does this:
# https://github.com/Zymrael/vortex/blob/f243e8ec5da8374be082a77056c0a447e7fa9231/vortex/model/tokenizer.py#L161C1-L161C9

from typing import List

def get_logits(sequence: str) -> torch.Tensor:
    r = requests.post(
        url="http://10.189.26.12:30100/biology/arc/evo2/forward",
        json={
            "sequence": sequence,
            "output_layers":["unembed"]
        },
    )
    response = json.loads(r.content.decode('utf-8'))
    if response.get("error", None):
        print(response["error"])
    data_string = response['data']

    decoded_data = base64.b64decode(data_string)
    buffer = io.BytesIO(decoded_data)
    npz_data = np.load(buffer)
    logits = np.array(npz_data['unembed.output'])
    return torch.tensor(logits)

def tokenize(seq) -> List:
    tokens = list(np.fromstring(seq, dtype=np.uint8))
    return tokens

def score_alt_nucleutide(sequences: List[str], reduce_method: str = 'mean', device='cpu'):

  # Tokenize
  input_ids = torch.tensor([tokenize(sequence) for sequence in sequences], dtype=torch.int64, device=device)

  # Get Logits
  with ThreadPoolExecutor(max_workers=2) as executor:
      logits_list = list(executor.map(get_logits, sequences))
      logits = torch.cat(logits_list, dim=0)  # n_batch_size x n_seq_length x 512
  
  # Apply Softmax
  softmax_logprobs = torch.log_softmax(logits, dim=-1)

  # Get likelihoods for the actual input sequences
  sequence_logprobs = torch.gather(
      softmax_logprobs,       # Gather likelihoods...
      2,                      # along the vocab dimension...
      input_ids.unsqueeze(-1)
  ).squeeze(-1)

#   return sequence_logprobs
  
  if reduce_method == 'sum': # PLL
     reduce_func = np.sum
  elif reduce_method == 'mean': # mean PLL
     reduce_func = np.mean
  else:
     raise ValueError(f'Invalid reduce_method {reduce_method}')

  return [
     reduce_func(sequence_logprobs[idx].cpu().numpy())
     for idx in range(len(sequences))
  ]

In [8]:
# clinvar_filtered_df = clinvar_df[clinvar_df['ALT'].str.contains(r'[ACTG]', regex=True)]
# clinvar_filtered_df['ALT'].unique()
# coding_1000_df = clinvar_filtered_df.sample(1000)
# final_combined_samples = pd.read_csv(f'{DATASETS_DIR}/clinvar_w_seq_202504_sampled_scored.csv')
final_combined_samples.shape

(1409, 34)

In [ ]:

# Assuming coding_df is your DataFrame and score_alt_nucleutide is defined
final_combined_samples['SEQ_SCORE'] = 0.0
final_combined_samples['ALT_SEQ_SCORE'] = 0.0

batch_size = 100
start_index = 0
for i in tqdm(range(start_index, len(final_combined_samples), batch_size)):
  rows = final_combined_samples.iloc[i:i+batch_size]
  final_combined_samples.loc[rows.index, 'SEQ_SCORE'] = score_alt_nucleutide(rows['SEQ'].values)
  final_combined_samples.loc[rows.index, 'ALT_SEQ_SCORE'] = score_alt_nucleutide(rows['ALT_SEQ'].values)
  final_combined_samples.to_csv(f'{DATASETS_DIR}/clinvar_w_seq_202504_sampled_scored.csv', index=False)

  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_3419383/2560354393.py:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  tokens = list(np.fromstring(seq, dtype=np.uint8))
/tmp/ipykernel_3419383/2560354393.py:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  tokens = list(np.fromstring(seq, dtype=np.uint8))
  7%|▋         | 1/15 [07:03<1:38:43, 423.09s/it]/tmp/ipykernel_3419383/2560354393.py:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  tokens = list(np.fromstring(seq, dtype=np.uint8))
/tmp/ipykernel_3419383/2560354393.py:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  tokens = list(np.fromstring(seq, dtype=np.uint8))
 13%|

In [ ]:
(final_combined_samples["SEQ_SCORE"] != 0).sum()
# coding_with_seq_df["ID"].iloc[0:1000]

ROCAUC

In [ ]:
from sklearn.metrics import roc_auc_score

final_combined_samples['PLL_SCORE'] = final_combined_samples['ALT_SEQ_SCORE'] - final_combined_samples['SEQ_SCORE']
roc_auc_score(final_combined_samples['INFO'], final_combined_samples['PLL_SCORE'])